In [1]:
%pip install yfinance
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install datetime
%pip install scikit-learn
%pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 7.6 MB/s eta 0:00:00


In [2]:
%pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=3386db76fd70d58f900e4dcd7873001c90c6722416961034eafa7a5d0c62b693
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta


## Load SP500 Data

In [6]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_ta
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import warnings

In [7]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

In [8]:
end_date = '2025-01-01'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

In [ ]:
df = yf.download(tickers=symbols_list, start=start_date, end=end_date, auto_adjust=False)

[***                    7%                       ]  35 of 503 completed

In [ ]:
df = df.stack()

## Calculate features and technical indicators for each stock

In [25]:
from typing_extensions import dataclass_transform
# Garman-Klass Volatility
df['garman_klass_vol']=((np.log(df['High']) - np.log(df['Low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['Adj Close']) - np.log(df['Open']))**2)

# RSI
df['rsi'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# BBL
df['bb_low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

In [27]:
# ATR
def compute_atr(stock_data):
  atr = pandas_ta.atr(high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], length=14)

  return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

In [54]:
# MACD
def compute_macd(close):
  macd = pandas_ta.macd(close=close, length=20).iloc[:,0]

  return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['Adj Close'].apply(compute_macd)

In [55]:
# Dollar Volume
df['dollar_volume'] = (df['Adj Close']*df['Volume'])/1e6

## Aggregate to montly level and filter top 150 most liquid stocks for each month

In [56]:
d_v = df.unstack('Ticker')['dollar_volume'].resample('ME').mean().stack('Ticker').to_frame('dollar_volume')

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'Volume', 'Open', 'High', 'Low', 'Close']]

others = df.unstack()[last_cols].resample('ME').last().stack('Ticker', future_stack=True)

idf = pd.concat([d_v, others], axis=1).dropna()

Calculate 5-year rolling average of dollar volume for each stocks before filtering

In [57]:
idf['dollar_volume'] = idf.loc[:,'dollar_volume'].unstack('Ticker').rolling(5*12).mean().stack()

idf['dollar_vol_rank'] = idf.groupby('Date')['dollar_volume'].rank(ascending=False)

idf = idf[idf['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

In [61]:
idf

Adj Close  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                      
2022-01-31 AAPL    171.549301          0.000482  56.861846  5.044068   
           ABBV    121.169334         -0.005636  64.192378  4.765774   
           ABT     119.732483         -0.000611  46.477958  4.721931   
           ACN     335.711121          0.000137  45.541324  5.708245   
           ADBE    534.299988          0.000440  46.113510  6.190550   
...                       ...               ...        ...       ...   
2024-12-31 WDAY    258.029999          0.000278  46.917931  5.552833   
           WFC      69.513557         -0.000007  49.517632  4.230842   
           WMT      89.885246          0.000025  50.924171  4.508084   
           XOM     105.643356          0.000187  35.592926  4.622929   
           ZTS     161.892609          0.000052  34.077556  5.068102   

                     bb_mid   bb_high       atr      macd  
Date       Ticker                                          
2022-01-31 AAPL    5.121558  5.199048  1.668394 -0.982385  
           ABBV    4.788866  4.811958  0.300582  0.690165  
           ABT     4.805731  4.889531  1.680831 -2.348231  
           ACN     5.834387  5.960529  2.833452 -2.913353  
           ADBE    6.254522  6.318495  2.322980 -2.018646  
...                     ...       ...       ...       ...  
2024-12-31 WDAY    5.598519  5.644205  1.193359  0.040827  
           WFC     4.272490  4.314138  1.151664 -0.322572  
           WMT     4.541688  4.575291  2.884244  0.622149  
           XOM     4.688257  4.753586  0.099058 -2.417724  
           ZTS     5.140142  5.212182  0.271221 -1.784608  

[5364 rows x 8 columns]

## Calculate Monthly Returns for different time horizons as features

In [64]:
def calculate_returns(df):

  outlier_cutoff = 0.005

  lags = [1, 2, 3, 6, 9, 12]

  for lag in lags:
    df[f'return_{lag}m'] = df['Adj Close'].pct_change(lag).pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                                              upper=x.quantile(1-outlier_cutoff))).add(1).pow(1/lag).sub(1)
  return df

idf = idf.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

In [65]:
idf

Adj Close  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                      
2023-01-31 AAPL    142.444824          0.000102  57.790457  4.809500   
           ABBV    135.760406         -0.001625  38.838123  4.874795   
           ABT     105.695557         -0.000425  52.707968  4.655936   
           ACN     268.963318         -0.000014  51.894259  5.543697   
           ADBE    370.339996          0.000075  61.957147  5.787025   
...                       ...               ...        ...       ...   
2024-12-31 WDAY    258.029999          0.000278  46.917931  5.552833   
           WFC      69.513557         -0.000007  49.517632  4.230842   
           WMT      89.885246          0.000025  50.924171  4.508084   
           XOM     105.643356          0.000187  35.592926  4.622929   
           ZTS     161.892609          0.000052  34.077556  5.068102   

                     bb_mid   bb_high       atr      macd  return_1m  \
Date       Ticker                                                      
2023-01-31 AAPL    4.904467  4.999434  0.657699  0.670478   0.110521   
           ABBV    4.952511  5.030227  0.316511 -1.883036  -0.077071   
           ABT     4.679652  4.703367  0.180814  0.284230   0.011481   
           ACN     5.585196  5.626696  0.811010 -0.119583   0.050035   
           ADBE    5.858914  5.930804 -0.077314  0.768984   0.100467   
...                     ...       ...       ...       ...        ...   
2024-12-31 WDAY    5.598519  5.644205  1.193359  0.040827   0.032161   
           WFC     4.272490  4.314138  1.151664 -0.322572  -0.077852   
           WMT     4.541688  4.575291  2.884244  0.622149  -0.021079   
           XOM     4.688257  4.753586  0.099058 -2.417724  -0.088081   
           ZTS     5.140142  5.212182  0.271221 -1.784608  -0.070300   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
Date       Ticker                                                          
2023-01-31 AAPL    -0.012713  -0.019532  -0.019126  -0.009294   -0.014606  
           ABBV    -0.038028   0.006233   0.008150   0.003874    0.009520  
           ABT      0.016002   0.039241   0.004144  -0.001407   -0.010338  
           ACN     -0.035073  -0.004373  -0.013989  -0.006812   -0.017697  
           ADBE     0.036179   0.051551  -0.016861  -0.007402   -0.030083  
...                      ...        ...        ...        ...         ...  
2024-12-31 WDAY     0.050428   0.018241   0.024187  -0.006145   -0.005613  
           WFC      0.043159   0.077381   0.030652   0.023750    0.032320  
           WMT      0.051163   0.038920   0.050136   0.047194    0.047235  
           XOM     -0.036308  -0.025576  -0.008568  -0.005896    0.008933  
           ZTS     -0.045357  -0.057651  -0.009506  -0.003353   -0.014816  

[3397 rows x 14 columns]

## Download Fama-French Factors and Calculate Rolling Factor Betas

In [84]:
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('ME').last().div(100)

factor_data = factor_data.join(idf['return_1m']).sort_index()

factor_data

<ipython-input-84-2290961161>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='2010')[0].drop('RF', axis=1)
<ipython-input-84-2290961161>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='2010')[0].drop('RF', axis=1)


Mkt-RF     SMB     HML     RMW     CMA  return_1m
Date       Ticker                                                   
2023-01-31 AAPL    0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.110521
           ABBV    0.0659  0.0441 -0.0392 -0.0244 -0.0441  -0.077071
           ABT     0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.011481
           ACN     0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.050035
           ADBE    0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.100467
...                   ...     ...     ...     ...     ...        ...
2024-12-31 WDAY   -0.0315 -0.0384 -0.0300  0.0191 -0.0121   0.032161
           WFC    -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.077852
           WMT    -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.021079
           XOM    -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.088081
           ZTS    -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.070300

[3397 rows x 6 columns]

Filter out stocks with less than 10 months of data

In [85]:
observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('Ticker').isin(valid_stocks.index)]

In [86]:
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
Date       Ticker                                                   
2023-01-31 AAPL    0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.110521
           ABBV    0.0659  0.0441 -0.0392 -0.0244 -0.0441  -0.077071
           ABT     0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.011481
           ACN     0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.050035
           ADBE    0.0659  0.0441 -0.0392 -0.0244 -0.0441   0.100467
...                   ...     ...     ...     ...     ...        ...
2024-12-31 VZ     -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.097548
           WDAY   -0.0315 -0.0384 -0.0300  0.0191 -0.0121   0.032161
           WFC    -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.077852
           WMT    -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.021079
           XOM    -0.0315 -0.0384 -0.0300  0.0191 -0.0121  -0.088081

[3344 rows x 6 columns]

Calculate Rolling Factor Betas

In [87]:
betas = factor_data.groupby(level=1, group_keys=False).apply(lambda x: RollingOLS(endog=x['return_1m'],
                                                                          exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                                                          window=min(24, x.shape[0]),
                                                                          min_nobs=len(x.columns)+1).fit(params_only=True).params.drop('const', axis=1))


In [ ]:
betas.shift()